CODIGO PARA FORMATEAR LAS COLUMNA DE FECHA Y 
POSTERIORMENTE VAMOS A HACER LIMPIEZA DE ESAS VARIABLES Y DIVIDIR EN GENERALES Y EN PISTA:
EN EL ULTIMO AÑO (2024+2025)
DESDE 2020 A 2023
PARA VER SU MEJORIA Y TENERLO EN CUENTA EN EL INFORME

In [1]:
import os
import pandas as pd


# Carpetas
input_folder = 'Tests/calidad_de_resto'
output_folder = 'Tests/calidad_de_resto_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")


AlexanderZverev_resto.csv procesado correctamente.
AndreyRublev_resto.csv procesado correctamente.
CarlosAlcaraz_resto.csv procesado correctamente.
CasperRuud_resto.csv procesado correctamente.
DaniilMedvedev_resto.csv procesado correctamente.
HolgerRune_resto.csv procesado correctamente.
HubertHurkacz_resto.csv procesado correctamente.
JannikSinner_resto.csv procesado correctamente.
NovakDjokovic_resto.csv procesado correctamente.
StefanosTsitsipas_resto.csv procesado correctamente.


In [11]:
# Carpeta con los CSVs
folder = 'Tests/calidad_de_resto_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_resto.csv
Match        0
RiP%         0
RiP W%       0
RetWnr%      0
FH/BH        0
RDI          0
Slice%       0
1st: RiP%    0
RiP W%.1     0
RetWnr%.1    0
RDI.1        0
Slice%.1     0
2nd: RiP%    0
RiP W%.2     0
RetWnr%.2    0
RDI.2        0
Slice%.2     0
W_or_L       0
Rival        0
dtype: int64

Nulos por columna en: AndreyRublev_resto.csv
Match        0
RiP%         0
RiP W%       0
RetWnr%      0
FH/BH        0
RDI          0
Slice%       0
1st: RiP%    0
RiP W%.1     0
RetWnr%.1    0
RDI.1        0
Slice%.1     0
2nd: RiP%    0
RiP W%.2     0
RetWnr%.2    0
RDI.2        0
Slice%.2     0
W_or_L       0
Rival        0
dtype: int64

Nulos por columna en: CarlosAlcaraz_resto.csv
Match        0
RiP%         0
RiP W%       0
RetWnr%      0
FH/BH        0
RDI          0
Slice%       0
1st: RiP%    0
RiP W%.1     0
RetWnr%.1    0
RDI.1        0
Slice%.1     0
2nd: RiP%    0
RiP W%.2     0
RetWnr%.2    0
RDI.2        0
Slice%.2     0
W_or_L 

In [3]:
# Carpetas
input_folder = 'Tests/calidad_rally'
output_folder = 'Tests/calidad_rally_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_rally.csv procesado correctamente.
AndreyRublev_rally.csv procesado correctamente.
CarlosAlcaraz_rally.csv procesado correctamente.
CasperRuud_rally.csv procesado correctamente.
DaniilMedvedev_rally.csv procesado correctamente.
HolgerRune_rally.csv procesado correctamente.
HubertHurkacz_rally.csv procesado correctamente.
JannikSinner_rally.csv procesado correctamente.
NovakDjokovic_rally.csv procesado correctamente.
StefanosTsitsipas_rally.csv procesado correctamente.


In [12]:
# Carpeta con los CSVs
folder = 'Tests/calidad_rally_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_rally.csv
Match          0
RallyLen       0
RLen-Serve     0
RLen-Return    0
1-3 W%         0
4-6 W%         0
7-9 W%         0
10+ W%         0
FH/GS          0
BH Slice%      0
FHP            0
FHP/100        0
BHP            0
BHP/100        0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: AndreyRublev_rally.csv
Match          0
RallyLen       0
RLen-Serve     0
RLen-Return    0
1-3 W%         0
4-6 W%         0
7-9 W%         0
10+ W%         0
FH/GS          0
BH Slice%      0
FHP            0
FHP/100        0
BHP            0
BHP/100        0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: CarlosAlcaraz_rally.csv
Match          0
RallyLen       0
RLen-Serve     0
RLen-Return    0
1-3 W%         0
4-6 W%         0
7-9 W%         0
10+ W%         0
FH/GS          0
BH Slice%      0
FHP            0
FHP/100        0
BHP            0
BHP/100        0
W_or_L         0
Rival          0
dtype: int64

Nu

In [4]:
# Carpetas
input_folder = 'Tests/calidad_servicio'
output_folder = 'Tests/calidad_servicio_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_servicio.csv procesado correctamente.
AndreyRublev_servicio.csv procesado correctamente.
CarlosAlcaraz_servicio.csv procesado correctamente.
CasperRuud_servicio.csv procesado correctamente.
DaniilMedvedev_servicio.csv procesado correctamente.
HolgerRune_servicio.csv procesado correctamente.
HubertHurkacz_servicio.csv procesado correctamente.
JannikSinner_servicio.csv procesado correctamente.
NovakDjokovic_servicio.csv procesado correctamente.
StefanosTsitsipas_servicio.csv procesado correctamente.


In [13]:
# Carpeta con los CSVs
folder = 'Tests/calidad_servicio_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_servicio.csv
Match          0
Unret%         0
<=3 W%         0
RiP W%         0
SvImpact       0
1st: Unret%    0
<=3 W%.1       0
RiP W%.1       0
SvImpact.1     0
D Wide%        0
A Wide%        0
BP Wide%       0
2nd: Unret%    0
<=3 W%.2       0
RiP W%.2       0
2ndAgg         0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: AndreyRublev_servicio.csv
Match          0
Unret%         0
<=3 W%         0
RiP W%         0
SvImpact       0
1st: Unret%    0
<=3 W%.1       0
RiP W%.1       0
SvImpact.1     0
D Wide%        0
A Wide%        0
BP Wide%       0
2nd: Unret%    0
<=3 W%.2       0
RiP W%.2       0
2ndAgg         0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: CarlosAlcaraz_servicio.csv
Match          0
Unret%         0
<=3 W%         0
RiP W%         0
SvImpact       0
1st: Unret%    0
<=3 W%.1       0
RiP W%.1       0
SvImpact.1     0
D Wide%        0
A Wide%        0
BP Wide%       0
2nd: Un

In [5]:
# Carpetas
input_folder = 'Tests/calidad_tactics'
output_folder = 'Tests/calidad_tactics_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_tactics.csv procesado correctamente.
AndreyRublev_tactics.csv procesado correctamente.
CarlosAlcaraz_tactics.csv procesado correctamente.
CasperRuud_tactics.csv procesado correctamente.
DaniilMedvedev_tactics.csv procesado correctamente.
HolgerRune_tactics.csv procesado correctamente.
HubertHurkacz_tactics.csv procesado correctamente.
JannikSinner_tactics.csv procesado correctamente.
NovakDjokovic_tactics.csv procesado correctamente.
StefanosTsitsipas_tactics.csv procesado correctamente.


In [14]:
# Carpeta con los CSVs
folder = 'Tests/calidad_tactics_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_tactics.csv
Match         0
SnV Freq      0
SnV W%        0
Net Freq      0
Net W%        0
FH: Wnr%      0
DTL Wnr%      0
IO Wnr%       0
BH: Wnr%      0
DTL Wnr%.1    0
Drop: Freq    0
Wnr%          0
RallyAgg      0
ReturnAgg     0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: AndreyRublev_tactics.csv
Match         0
SnV Freq      0
SnV W%        0
Net Freq      0
Net W%        0
FH: Wnr%      0
DTL Wnr%      0
IO Wnr%       0
BH: Wnr%      0
DTL Wnr%.1    0
Drop: Freq    0
Wnr%          0
RallyAgg      0
ReturnAgg     0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: CarlosAlcaraz_tactics.csv
Match         0
SnV Freq      0
SnV W%        0
Net Freq      0
Net W%        0
FH: Wnr%      0
DTL Wnr%      0
IO Wnr%       0
BH: Wnr%      0
DTL Wnr%.1    0
Drop: Freq    0
Wnr%          0
RallyAgg      0
ReturnAgg     0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: CasperRuud_tactics.csv

In [6]:
# Carpetas
input_folder = 'Tests/key_games_results'
output_folder = 'Tests/key_games_results_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_key_games.csv procesado correctamente.
AndreyRublev_key_games.csv procesado correctamente.
CarlosAlcaraz_key_games.csv procesado correctamente.
CasperRuud_key_games.csv procesado correctamente.
DaniilMedvedev_key_games.csv procesado correctamente.
HolgerRune_key_games.csv procesado correctamente.
HubertHurkacz_key_games.csv procesado correctamente.
JannikSinner_key_games.csv procesado correctamente.
NovakDjokovic_key_games.csv procesado correctamente.
StefanosTsitsipas_key_games.csv procesado correctamente.


In [15]:
# Carpeta con los CSVs
folder = 'Tests/key_games_results_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_key_games.csv
Match          0
BP Games       0
BP Conv/BPG    0
BreakBack%     0
BPF Games      0
Hold/BPFG      0
Consol%        0
SvForSet       0
SvStaySet      0
SvForMatch     0
SvStayMatch    0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: AndreyRublev_key_games.csv
Match          0
BP Games       0
BP Conv/BPG    0
BreakBack%     0
BPF Games      0
Hold/BPFG      0
Consol%        0
SvForSet       0
SvStaySet      0
SvForMatch     0
SvStayMatch    0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: CarlosAlcaraz_key_games.csv
Match          0
BP Games       0
BP Conv/BPG    0
BreakBack%     0
BPF Games      0
Hold/BPFG      0
Consol%        0
SvForSet       0
SvStaySet      0
SvForMatch     0
SvStayMatch    0
W_or_L         0
Rival          0
dtype: int64

Nulos por columna en: CasperRuud_key_games.csv
Match          0
BP Games       0
BP Conv/BPG    0
BreakBack%     0
BPF Games      0
Hold/BPFG  

In [7]:
# Carpetas
input_folder = 'Tests/key_points_results'
output_folder = 'Tests/key_points_results_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_key_points.csv procesado correctamente.
AndreyRublev_key_points.csv procesado correctamente.
CarlosAlcaraz_key_points.csv procesado correctamente.
CasperRuud_key_points.csv procesado correctamente.
DaniilMedvedev_key_points.csv procesado correctamente.
HolgerRune_key_points.csv procesado correctamente.
HubertHurkacz_key_points.csv procesado correctamente.
JannikSinner_key_points.csv procesado correctamente.
NovakDjokovic_key_points.csv procesado correctamente.
StefanosTsitsipas_key_points.csv procesado correctamente.


In [16]:
# Carpeta con los CSVs
folder = 'Tests/key_points_results_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_key_points.csv
Match       0
GP Conv     0
BP Conv     0
SP Conv     0
MP Conv     0
BP Saved    0
SP Saved    0
MP Saved    0
TB SPW      0
TB RPW      0
W_or_L      0
Rival       0
dtype: int64

Nulos por columna en: AndreyRublev_key_points.csv
Match       0
GP Conv     0
BP Conv     0
SP Conv     0
MP Conv     0
BP Saved    0
SP Saved    0
MP Saved    0
TB SPW      0
TB RPW      0
W_or_L      0
Rival       0
dtype: int64

Nulos por columna en: CarlosAlcaraz_key_points.csv
Match       0
GP Conv     0
BP Conv     0
SP Conv     0
MP Conv     0
BP Saved    0
SP Saved    0
MP Saved    0
TB SPW      0
TB RPW      0
W_or_L      0
Rival       0
dtype: int64

Nulos por columna en: CasperRuud_key_points.csv
Match       0
GP Conv     0
BP Conv     0
SP Conv     0
MP Conv     0
BP Saved    0
SP Saved    0
MP Saved    0
TB SPW      0
TB RPW      0
W_or_L      0
Rival       0
dtype: int64

Nulos por columna en: DaniilMedvedev_key_points.csv
Match       0
GP 

In [8]:
# Carpetas
input_folder = 'Tests/point_by_point_stats_results'
output_folder = 'Tests/point_by_point_stats_results_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_pbp_stats.csv procesado correctamente.
AndreyRublev_pbp_stats.csv procesado correctamente.
CarlosAlcaraz_pbp_stats.csv procesado correctamente.
CasperRuud_pbp_stats.csv procesado correctamente.
DaniilMedvedev_pbp_stats.csv procesado correctamente.
HolgerRune_pbp_stats.csv procesado correctamente.
HubertHurkacz_pbp_stats.csv procesado correctamente.
JannikSinner_pbp_stats.csv procesado correctamente.
NovakDjokovic_pbp_stats.csv procesado correctamente.
StefanosTsitsipas_pbp_stats.csv procesado correctamente.


In [17]:
# Carpeta con los CSVs
folder = 'Tests/point_by_point_stats_results_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_pbp_stats.csv
Match         0
BLR           0
DR+           0
EI            0
CBF           0
Deuce A%      0
Deuce SPW%    0
Ad A%         0
Ad SPW%       0
Deuce RPW%    0
Ad RPW%       0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: AndreyRublev_pbp_stats.csv
Match         0
BLR           0
DR+           0
EI            0
CBF           0
Deuce A%      0
Deuce SPW%    0
Ad A%         0
Ad SPW%       0
Deuce RPW%    0
Ad RPW%       0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: CarlosAlcaraz_pbp_stats.csv
Match         0
BLR           0
DR+           0
EI            0
CBF           0
Deuce A%      0
Deuce SPW%    0
Ad A%         0
Ad SPW%       0
Deuce RPW%    0
Ad RPW%       0
W_or_L        0
Rival         0
dtype: int64

Nulos por columna en: CasperRuud_pbp_stats.csv
Match         0
BLR           0
DR+           0
EI            0
CBF           0
Deuce A%      0
Deuce SPW%    0
Ad A%         0
Ad SPW%

In [9]:
# Carpetas
input_folder = 'Tests/winners_errors_results'
output_folder = 'Tests/winners_errors_results_limpio'
os.makedirs(output_folder, exist_ok=True)

# Procesar cada CSV en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_path = os.path.join(input_folder, filename)
        df = pd.read_csv(input_path)

        if 'Result' not in df.columns:
            print(f"{filename}: no se encontró columna 'Result', se omite.")
            continue

        # Limpiar columna Result y dividirla
        df['Result'] = df['Result'].astype(str).str.strip()
        df['W_or_L'] = df['Result'].str[0]
        df['Rival'] = df['Result'].str.extract(r'vs(.+)', expand=False).str.strip()

        # Eliminar columna original
        df = df.drop(columns=['Result'])

        # Guardar nuevo CSV
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
        print(f"{filename} procesado correctamente.")

AlexanderZverev_winners_errors.csv procesado correctamente.
AndreyRublev_winners_errors.csv procesado correctamente.
CarlosAlcaraz_winners_errors.csv procesado correctamente.
CasperRuud_winners_errors.csv procesado correctamente.
DaniilMedvedev_winners_errors.csv procesado correctamente.
HolgerRune_winners_errors.csv procesado correctamente.
HubertHurkacz_winners_errors.csv procesado correctamente.
JannikSinner_winners_errors.csv procesado correctamente.
NovakDjokovic_winners_errors.csv procesado correctamente.
StefanosTsitsipas_winners_errors.csv procesado correctamente.


In [ ]:
# Carpeta con los CSVs
folder = 'Tests/winners_errors_results_limpio'

# Revisar cada archivo
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        path = os.path.join(folder, filename)
        df = pd.read_csv(path)

        print(f"\nNulos por columna en: {filename}")
        print(df.isnull().sum())


Nulos por columna en: AlexanderZverev_winners_errors.csv
Match            0
Winners          0
UFEs             0
Ratio            0
Wnr/Pt           0
UFE/Pt           0
RallyWinners     0
RallyUFEs        0
RallyRatio       0
Rally Wnr/Pt     0
Rally UFE/Pt     0
FH Wnr/Pt       34
BH Wnr/Pt       34
vs Ratio         0
vs Wnr/Pt        0
vs UFE/Pt        0
W_or_L           0
Rival            0
dtype: int64

Nulos por columna en: AndreyRublev_winners_errors.csv
Match            0
Winners          0
UFEs             0
Ratio            0
Wnr/Pt           0
UFE/Pt           0
RallyWinners     0
RallyUFEs        0
RallyRatio       0
Rally Wnr/Pt     0
Rally UFE/Pt     0
FH Wnr/Pt       18
BH Wnr/Pt       18
vs Ratio         0
vs Wnr/Pt        0
vs UFE/Pt        0
W_or_L           0
Rival            0
dtype: int64

Nulos por columna en: CarlosAlcaraz_winners_errors.csv
Match           0
Winners         0
UFEs            0
Ratio           0
Wnr/Pt          0
UFE/Pt          0
RallyWinners 

: 

In [9]:
import os
import pandas as pd

# Configuración
carpeta_entrada = 'Tests_limpio/calidad_de_resto_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/calidad_de_resto3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")


Conversión completada.


In [11]:
# Configuración
carpeta_entrada = 'Tests_limpio/calidad_rally_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/calidad_rally3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [13]:
# Configuración
carpeta_entrada = 'Tests_limpio/calidad_servicio_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/calidad_servicio3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [15]:
# Configuración
carpeta_entrada = 'Tests_limpio/calidad_tactics_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/calidad_tactics3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [17]:
# Configuración
carpeta_entrada = 'Tests_limpio/key_games_results_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/key_games_results3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [19]:
# Configuración
carpeta_entrada = 'Tests_limpio/key_points_results_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/key_points_results3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [21]:
# Configuración
carpeta_entrada = 'Tests_limpio/point_by_point_stats_results_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/point_by_point_stats_results3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [23]:
# Configuración
carpeta_entrada = 'Tests_limpio/winners_errors_results_limpio'         # Carpeta con CSVs originales
carpeta_salida = 'Test_3/winners_errors_results3'     # Carpeta para guardar los nuevos CSVs

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para dividir la columna "Match"
def dividir_match(valor):
    partes = valor.strip().split()
    if len(partes) < 3:
        return pd.Series([None, None, None])  # Maneja casos incorrectos
    date = partes[0]
    rd = partes[-1]
    tournament = ' '.join(partes[1:-1])
    return pd.Series([date, tournament, rd])

# Procesar cada archivo CSV
for archivo in os.listdir(carpeta_entrada):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_entrada, archivo)
        df = pd.read_csv(ruta_csv)

        # Aplicar la división de la columna "Match"
        df[['Date', 'Tournament', 'Rd']] = df['Match'].apply(dividir_match)

        # Eliminar la columna original "Match"
        df = df.drop(columns=['Match'])

        # (Opcional) Mover las nuevas columnas al principio
        columnas_ordenadas = ['Date', 'Tournament', 'Rd'] + [col for col in df.columns if col not in ['Date', 'Tournament', 'Rd']]
        df = df[columnas_ordenadas]

        # Guardar el CSV modificado
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Conversión completada.")

Conversión completada.


In [24]:


# Procesar todos los archivos CSV
for archivo in os.listdir(carpeta_salida):
    if archivo.endswith('.csv'):
        ruta_csv = os.path.join(carpeta_salida, archivo)
        df = pd.read_csv(ruta_csv)



        # Reordenar columnas: mover la última a la posición 4
        cols = df.columns.tolist()
        last_col = cols[-1]
        cols.remove(last_col)
        cols.insert(3, last_col)  # Insertar en la posición 4 (5ta columna)
        df = df[cols]

        # Guardar
        ruta_salida = os.path.join(carpeta_salida, archivo)
        df.to_csv(ruta_salida, index=False)

print("Columnas reorganizadas. Archivos guardados en:", carpeta_salida)

Columnas reorganizadas. Archivos guardados en: Test_3/winners_errors_results3
